In [1]:
import rospy
from thr_infrastructure_msgs.srv import GetCarryPose, GetCarryPoseRequest
import tf
import os
import transformations

In [2]:
rospy.init_node('create_grid')

In [3]:
srv = rospy.ServiceProxy('thr/get_carry_pose', GetCarryPose)
rospy.wait_for_service('thr/get_carry_pose')
tfl = tf.TransformListener()

KeyboardInterrupt: 

In [13]:
ObjectTGripper = tfl.lookupTransform('shapeo', 'right_gripper', rospy.Time(0))

In [14]:
ObjectTGripper

((-0.039633793548917884, -0.0058272953891080614, 0.06672552362895134),
 (0.7284606267128245,
  0.6053606642424952,
  -0.12075812581014464,
  0.29715493697373624))

## Defintion of functions

In [17]:
def get_handover_location(method):
    req = GetCarryPoseRequest()
    req.object = '/shapeo'
    req.laterality = 'right'
    req.location = 'ellipse'
    res = srv(req)
    # express the result in VREP frame
    VrepTBase = tfl.lookupTransform('vrep_frame', 'base', rospy.Time(0))
    GripperTVrep = tfl.lookupTransform('right_gripper', 'vrep_tip', rospy.Time(0))
    # apply chain rule of transformation
    robot_pose = transformations.multiply_transform(VrepTBase, res.object_pose)
    robot_pose = transformations.multiply_transform(robot_pose, ObjectTGripper)
    robot_pose = transformations.multiply_transform(robot_pose, GripperTVrep)
    return robot_pose

In [16]:
def convert_to_string(pose):
    # create the string of the pose to write in the textfile
    pose_str = ''
    for lists in pose:
        for value in lists:
            pose_str += str(value) + '\t' 
    pose_str += '\n'
    return pose_str

## Create table of hip positions 

In [6]:
hip_corners = []
# get the 4 corners of the grid by asking the human to move between them
for i in range(4):
    rospy.sleep(5)
    hip_corners.append(get_handover_location('reba'))
    hip_corners.append(get_handover_location('relative'))
    os.system('beep')
# keep the same rotation
rotation = hip_corners[0][1] 

In [7]:
hip_pose_grid = []
# fill the hip grid by interpolation
nb_cols = 10
nb_rows = 4
l1 = linear_3d_interpolation(hip_corners[0][0], hip_corners[2][0], nb_rows)
l2 = linear_3d_interpolation(hip_corners[1][0], hip_corners[3][0], nb_rows)
for i in range(nb_rows):
    row = linear_3d_interpolation(l1[i], l2[i], nb_cols)
    hip_pose_grid.append([])
    for trans in row:
        hip_pose_grid[i].append([trans,rotation])

## Calculate corresponding handover table

In [8]:
hand_over_grid = []
for i in range(2):
    hand_over_grid.append([])
    for hip_vectors in hip_pose_grid:
        for hip_poses in hip_vectors:
            pose = get_handover_location(hip_poses, reba_service, i)
            hand_over_grid[i].append(transformations.pose_to_list(pose))

## Pulbish them to check result in Rviz

In [9]:
tfb = tf.TransformBroadcaster()

In [11]:
side = 0
rate = rospy.Rate(10)
while not rospy.is_shutdown():
    for i in range(len(hand_over_grid[side])):
        tfb.sendTransform(hand_over_grid[side][i][0],hand_over_grid[side][i][1],rospy.Time.now(),'/vrep/handover/'+str(i),'/vrep_frame')
    rate.sleep()

KeyboardInterrupt: 

## Write result to a text file

In [12]:
pose_str = ''
for pose in hand_over_grid[0]:
    pose_str += convert_to_string(pose)
# write the string to the file
with open('/tmp/reba_table_left.txt', 'w') as f:
    f.write(pose_str)

In [13]:
pose_str = ''
for pose in hand_over_grid[1]:
    pose_str += convert_to_string(pose)
# write the string to the file
with open('/tmp/reba_table_right.txt', 'w') as f:
    f.write(pose_str)